# <a id='toc1_'></a>[Data Cleaning and Processing](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Data Cleaning Processes](#toc2_)    
  - [Removing Unnecessary Columns](#toc2_1_)    
  - [NaN Values in the Data](#toc2_2_)    
  - [Duplicates in the Data](#toc2_3_)    
  - [Outliers in the Data](#toc2_4_)    
  - [Standardizing and Checking Data Types](#toc2_5_)    
  - [Dealing with Text Data](#toc2_6_)    
  - [Dealing with Categorical Data](#toc2_7_)    
  - [Checking for Data Consistency](#toc2_8_)    
  - [Normalizing Data](#toc2_9_)    
  - [Check Data Imbalance](#toc2_10_)    
  - [Feature Engineering](#toc2_11_)    
- [Creating New Dataset and Saving](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [114]:
# import all the necessary packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re


In this workbook we will clean and process the data for the project. We will also create a new dataset that will be used for the analysis. This dataset will be saved as a csv file and will be used in the analysis workbook. Specifically, we will do the following:

1. Load the data from the csv file created in the data loadings workbook
2. Clean the data by:
    - removing or looking at columns that are not needed
    - looking at missing values
    - looking at duplicates
    - looking at outliers
    - standardizing and checking data types
    - deal with text data
    - deal with categorical data (categories)
    - check for data consistency across columns
    - look at normalizing data (price, votes, etc. )
    - Looking at rows with missing values
    - check data balance
    - feature engineering (sentiment analysis, etc.)

3. Create a new dataset
4. Save the new dataset

# <a id='toc2_'></a>[Data Cleaning Processes](#toc0_)

In [115]:
# load data - MAC OS
amz_rev = pd.read_csv('/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/all_revs_meta.csv', low_memory=True)


In [116]:
#  drop column 'Unnamed: 0'
amz_rev = amz_rev.drop(columns=['Unnamed: 0'])

# initial data view
display(amz_rev.head(3))
print("Shape of the data: ", amz_rev.shape)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,category_x,vote,description,title,brand,feature,rank,main_cat,date,price
0,5.0,False,"04 7, 2018",A31URN5S2Q0UJV,B000URXP6E,{'Size:': ' Small'},Boris Jones,Was skeptical at first. The liquid is kind of ...,Awesome quality!,1523059200,beauty,NaN,['Juicy burst of Starburst fruit flavored Lip ...,Bonne Bell Smackers Bath and Body Starburst Co...,Bonne Bell,[],"1,390,827 in Beauty &amp; Personal Care (",All Beauty,NaN,NaN
1,5.0,True,"02 4, 2014",A31XUJMEDBUGKR,B000URXP6E,{'Size:': ' 23'},Terry V.,Beautiful Beginnings have been the answer to m...,Works great!,1391472000,beauty,NaN,['Juicy burst of Starburst fruit flavored Lip ...,Bonne Bell Smackers Bath and Body Starburst Co...,Bonne Bell,[],"1,390,827 in Beauty &amp; Personal Care (",All Beauty,NaN,NaN
2,5.0,True,"05 11, 2013",A2XPTXCAX8WLHU,B000URXP6E,{'Size:': ' 263'},Mindy Lipton,My daughter bought this for me because she kno...,Love it,1368230400,beauty,NaN,['Juicy burst of Starburst fruit flavored Lip ...,Bonne Bell Smackers Bath and Body Starburst Co...,Bonne Bell,[],"1,390,827 in Beauty &amp; Personal Care (",All Beauty,NaN,NaN


Shape of the data:  (617770, 20)


## <a id='toc2_1_'></a>[Removing Unnecessary Columns](#toc0_)

We look at removing columns that are not needed for the analysis.


In [117]:
# see columns
print("Columns in the data: \n", amz_rev.columns)

Columns in the data: 
 Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'category_x',
       'vote', 'description', 'title', 'brand', 'feature', 'rank', 'main_cat',
       'date', 'price'],
      dtype='object')


In [118]:
# remove certain columns
amz_rev.drop(['verified', 'style', 'reviewTime', 'vote', 'feature', 'main_cat', 'date', 'summary', 'price', 'rank'], axis=1, inplace=True)

# see updated dataframe
display(amz_rev.head(3))

,overall,reviewerID,asin,reviewerName,reviewText,unixReviewTime,category_x,description,title,brand
0,5.0,A31URN5S2Q0UJV,B000URXP6E,Boris Jones,Was skeptical at first. The liquid is kind of ...,1523059200,beauty,['Juicy burst of Starburst fruit flavored Lip ...,Bonne Bell Smackers Bath and Body Starburst Co...,Bonne Bell
1,5.0,A31XUJMEDBUGKR,B000URXP6E,Terry V.,Beautiful Beginnings have been the answer to m...,1391472000,beauty,['Juicy burst of Starburst fruit flavored Lip ...,Bonne Bell Smackers Bath and Body Starburst Co...,Bonne Bell
2,5.0,A2XPTXCAX8WLHU,B000URXP6E,Mindy Lipton,My daughter bought this for me because she kno...,1368230400,beauty,['Juicy burst of Starburst fruit flavored Lip ...,Bonne Bell Smackers Bath and Body Starburst Co...,Bonne Bell


In [119]:
# sort and order the data
amz_rev.sort_values(by=['asin', 'overall'], ascending=[True, False], inplace=True)

# reorder columns
amz_rev = amz_rev[['reviewerID', 'reviewerName', 'unixReviewTime', 'asin', 'title', 'brand', 'description', 'reviewText', 'category_x', 'overall']]

# see updated dataframe
display(amz_rev.head(3))

,reviewerID,reviewerName,unixReviewTime,asin,title,brand,description,reviewText,category_x,overall
563599,A3WJELEV137U,John Bennett,1433462400,0001526863,Steve Green: Hide 'em in Your Heart: 13 Bible ...,Steve Green,"[""Steve Green: Hide 'em in Your Heart: 13 Bibl...",Product worked as advertised and am pleased wi...,movies_and_tv,5.0
539385,A2P64DWIPY4O2,Joycemarie58,1357430400,0005019281,An American Christmas Carol VHS,NaN,"[""In Depression-era New England, a miserly bus...",This is one of my favorites... I was very happ...,movies_and_tv,5.0
554411,ACER35NN2GG52,Jack2,1482710400,0005019281,An American Christmas Carol VHS,NaN,"[""In Depression-era New England, a miserly bus...",Interesting adaption of the scrooge story.,movies_and_tv,3.0


## <a id='toc2_2_'></a>[NaN/Missing Values in the Data](#toc0_)

We look at NaN values in the data. If any review (row) has missing values (NaN) in any column listed here:
    - (description, title, brand, price, rank)
then we remove that review (row) from the dataset. 



In [120]:
# how many rows have all null values in columns: description, title, brand
print("Number of rows with all null values in columns: description, title, brand: ", amz_rev[(amz_rev['description'].isnull()) & (amz_rev['title'].isnull()) & (amz_rev['brand'].isnull())].shape[0])

# see some of the rows with all null values in columns: description, title, brand, price, rank
display(amz_rev[(amz_rev['description'].isnull()) & (amz_rev['title'].isnull()) & (amz_rev['brand'].isnull())].head(3))

# which categories of reviews have the most null values in columns: description, title, brand, price, rank
amz_rev[(amz_rev['description'].isnull()) & (amz_rev['title'].isnull()) & (amz_rev['brand'].isnull())].groupby(['category_x']).size().sort_values(ascending=False)

Number of rows with all null values in columns: description, title, brand:  28168


,reviewerID,reviewerName,unixReviewTime,asin,title,brand,description,reviewText,category_x,overall
128638,ACWMEEUINNGEP,Sean,1504656000,0209688726,NaN,NaN,NaN,"Nifty little vac/pump, I absolutely love the f...",automotive,3.0
103788,A1NTV9VYC23VGY,Kindle Customer,1513555200,0449819906,NaN,NaN,NaN,"So easy to understand, great fun learning new ...",arts_crafts,5.0
107475,A304F5HPIBLUX2,dkp,1434758400,048625531X,NaN,NaN,NaN,great for kids,arts_crafts,5.0


category_x
digital_music                 24646
cds_and_vinyl                  2647
arts_crafts                     105
kindle_store                    103
tools_and_home_improvement       97
video_games                      71
magazine_subscriptions           64
sports_and_outdoors              48
patio_lawn_and_garden            48
toys_and_games                   40
home_and_kitchen                 39
industrial                       39
luxury_beauty                    36
automotive                       36
movies_and_tv                    34
office_products                  30
pet_supplies                     24
electronics                      23
grocery_and_gourmet_food         19
musical_instruments              10
cell_phones                       4
prime_pantry                      3
appliances                        2
dtype: int64

In [121]:
# remove rows with all null values in columns: description, title, brand
amz_rev = amz_rev[~((amz_rev['description'].isnull()) & (amz_rev['title'].isnull()) & (amz_rev['brand'].isnull()))]

# see updated dataframe
display(amz_rev.head(3))

# see value counts for column: category_x
amz_rev['category_x'].value_counts()

,reviewerID,reviewerName,unixReviewTime,asin,title,brand,description,reviewText,category_x,overall
563599,A3WJELEV137U,John Bennett,1433462400,0001526863,Steve Green: Hide 'em in Your Heart: 13 Bible ...,Steve Green,"[""Steve Green: Hide 'em in Your Heart: 13 Bibl...",Product worked as advertised and am pleased wi...,movies_and_tv,5.0
539385,A2P64DWIPY4O2,Joycemarie58,1357430400,0005019281,An American Christmas Carol VHS,NaN,"[""In Depression-era New England, a miserly bus...",This is one of my favorites... I was very happ...,movies_and_tv,5.0
554411,ACER35NN2GG52,Jack2,1482710400,0005019281,An American Christmas Carol VHS,NaN,"[""In Depression-era New England, a miserly bus...",Interesting adaption of the scrooge story.,movies_and_tv,3.0


musical_instruments           31134
office_products               28545
video_games                   28525
movies_and_tv                 28218
pet_supplies                  27795
cds_and_vinyl                 27114
tools_and_home_improvement    26672
industrial                    26495
home_and_kitchen              26237
electronics                   26197
luxury_beauty                 26098
patio_lawn_and_garden         26072
automotive                    25908
toys_and_games                25806
grocery_and_gourmet_food      25481
clothing_shoes_and_jewelry    25383
sports_and_outdoors           25323
arts_crafts                   25287
cell_phones                   25011
prime_pantry                  24997
kindle_store                  24897
software                      14103
beauty                         5767
magazine_subscriptions         3746
fashion                        3176
gift_cards                     2972
appliances                     2281
digital_music               

In [122]:
# remove rows with null values in column: description
amz_rev = amz_rev[~(amz_rev['description'].isnull())]

# remove rows with null values in column: reviewText
amz_rev = amz_rev[~(amz_rev['reviewText'].isnull())]

# remove rows with null values in column: title
amz_rev = amz_rev[~(amz_rev['title'].isnull())]

# remove rows with null values in column: brand
amz_rev = amz_rev[~(amz_rev['brand'].isnull())]

In [123]:
# shape of the data
print("Shape of the data: ", amz_rev.shape)

# see if any missing values
print("\nNumber of missing values: ", amz_rev.isnull().sum().sum())

# show count columns have missing values (as a percentage)
print("\Percentage of missing values in each column: \n", round(amz_rev.isnull().sum()/len(amz_rev)*100,2))

# show count of rows with missing data per category (as a percentage)
print("\Percentage of missing values in each category: \n", round(amz_rev.groupby(['category_x']).apply(lambda x: x.isnull().sum()).sum(axis=1)/len(amz_rev)*100,2))

Shape of the data:  (538515, 10)

Number of missing values:  101
\Percentage of missing values in each column: 
 reviewerID        0.00
reviewerName      0.02
unixReviewTime    0.00
asin              0.00
title             0.00
brand             0.00
description       0.00
reviewText        0.00
category_x        0.00
overall           0.00
dtype: float64
\Percentage of missing values in each category: 
 category_x
appliances                    0.0
arts_crafts                   0.0
automotive                    0.0
beauty                        0.0
cds_and_vinyl                 0.0
cell_phones                   0.0
clothing_shoes_and_jewelry    0.0
digital_music                 0.0
electronics                   0.0
fashion                       0.0
gift_cards                    0.0
grocery_and_gourmet_food      0.0
home_and_kitchen              0.0
industrial                    0.0
kindle_store                  0.0
luxury_beauty                 0.0
magazine_subscriptions        0.0
mov

## <a id='toc2_3_'></a>[Duplicates in the Data](#toc0_)

We looking at duplicates in the data.

A duplicate is defined as a review (row) that has the same values across all columns. We remove duplicates from the dataset.

In [124]:
# see if any duplicates
print("Number of duplicates: ", amz_rev.duplicated().sum())

# see duplicates
amz_rev[amz_rev.duplicated(keep=False)].sort_values(by=['asin']).head(3)

Number of duplicates:  42257


,reviewerID,reviewerName,unixReviewTime,asin,title,brand,description,reviewText,category_x,overall
142104,A1LICUG9QWBN4N,M. L. Johnson,1185667200,B000000J04,Ray Gun Suitcase,Pere Ubu,[],After tinkering with a more (supposedly) comme...,cds_and_vinyl,5.0
150133,A1LICUG9QWBN4N,M. L. Johnson,1185667200,B000000J04,Ray Gun Suitcase,Pere Ubu,[],After tinkering with a more (supposedly) comme...,cds_and_vinyl,5.0
140859,A1LO9CD1U3DSO8,Pancake,1359936000,B00000I8A3,Singin' in the Bathtub,John Lithgow,['CD'],My kids love this. Its quite entertaining for...,cds_and_vinyl,5.0


In [125]:
# remove duplicates
amz_rev.drop_duplicates(inplace=True)

# see updated dataframe
display(amz_rev.head(3))

# shape of the data
print("Shape of the data: ", amz_rev.shape)

,reviewerID,reviewerName,unixReviewTime,asin,title,brand,description,reviewText,category_x,overall
563599,A3WJELEV137U,John Bennett,1433462400,0001526863,Steve Green: Hide 'em in Your Heart: 13 Bible ...,Steve Green,"[""Steve Green: Hide 'em in Your Heart: 13 Bibl...",Product worked as advertised and am pleased wi...,movies_and_tv,5.0
162328,A3OJM6TXMK3J53,samson,1449273600,0005164885,Christmas Eve and Other Stories,Trans-Siberian Orchestra,"['This is a concept album all the way, with ta...",nice to hear this cd beautiful,cds_and_vinyl,5.0
147642,A14YR7QK3ASFFW,Amazon Customer,988070400,0005465079,Forgiven,Don Francisco,"[""1. Jesus Lord Of The Way I Feel, 2. Jehoshap...",That one song has to be one of the best in CCM...,cds_and_vinyl,5.0


Shape of the data:  (496258, 10)


## <a id='toc2_4_'></a>[Outliers in the Data](#toc0_)

We looking at outliers in the data.

In [126]:
# get summary statistics for the data - overall
amz_rev['overall'].describe()

count    496258.000000
mean          4.383577
std           1.086635
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: overall, dtype: float64

## <a id='toc2_5_'></a>[Standardizing and Checking Data Types](#toc0_)

We turn to standardizing and checking data types. 

In [127]:
# change unixReviewTime to datetime
amz_rev['unixReviewTime'] = pd.to_datetime(amz_rev['unixReviewTime'], unit='s')

# rename column: unixReviewTime to reviewTime
amz_rev.rename(columns={'unixReviewTime': 'reviewTime'}, inplace=True)

# see updated dataframe
display(amz_rev.head(3))

,reviewerID,reviewerName,reviewTime,asin,title,brand,description,reviewText,category_x,overall
563599,A3WJELEV137U,John Bennett,2015-06-05,0001526863,Steve Green: Hide 'em in Your Heart: 13 Bible ...,Steve Green,"[""Steve Green: Hide 'em in Your Heart: 13 Bibl...",Product worked as advertised and am pleased wi...,movies_and_tv,5.0
162328,A3OJM6TXMK3J53,samson,2015-12-05,0005164885,Christmas Eve and Other Stories,Trans-Siberian Orchestra,"['This is a concept album all the way, with ta...",nice to hear this cd beautiful,cds_and_vinyl,5.0
147642,A14YR7QK3ASFFW,Amazon Customer,2001-04-24,0005465079,Forgiven,Don Francisco,"[""1. Jesus Lord Of The Way I Feel, 2. Jehoshap...",That one song has to be one of the best in CCM...,cds_and_vinyl,5.0


In [128]:
# check data types
amz_rev.dtypes

reviewerID              object
reviewerName            object
reviewTime      datetime64[ns]
asin                    object
title                   object
brand                   object
description             object
reviewText              object
category_x              object
overall                float64
dtype: object

## <a id='toc2_6_'></a>[Dealing with Text Data](#toc0_)

We deal with text data. We have the following columns that contain text data:
- description
- title
- brand
- category
- reviewText


We handle each of these columns separately. We look out for: 

**Identify special characters or symbols**: Look for any special characters, symbols, or non-alphanumeric characters that may need to be cleaned or removed. These characters can sometimes interfere with downstream analysis or modeling.

**Handle HTML tags or formatting**: If the 'description' column contains HTML tags or formatting, you may consider removing them or converting them into plain text

### Description

In [129]:
# description column
print("Before cleaning:", amz_rev['description'][0])

# create function to clean description column
def clean_description(text):
    # Remove square brackets, single quotes, HTML tags, double quotes, commas, URLs, and CSS styling
    text = re.sub(r"\[|\]|\'|<[^>]*>|\"|,|\\https?://[^\s]+|{[^}]+}", "", text)
    
    # Remove extra whitespace
    text = " ".join(text.split())
    
    # Remove trailing punctuation
    text = re.sub(r"\.$", "", text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# apply function to description column
amz_rev['description'] = amz_rev['description'].apply(clean_description)

# see updated dataframe
print("\nAfter cleaning:", amz_rev['description'][0])



Before cleaning: ['Juicy burst of Starburst fruit flavored Lip Smacker lip glosses. Smooth formula glides on lips for moisture and juicy flavor that lasts. Fun lip smacker flavors in limited collection  are offered every year, especially during holidays.', '', '']

After cleaning: juicy burst of starburst fruit flavored lip smacker lip glosses. smooth formula glides on lips for moisture and juicy flavor that lasts. fun lip smacker flavors in limited collection are offered every year especially during holidays


In [130]:
# see which products have "http" in their description
amz_rev[amz_rev['description'].str.contains("http")]['asin'].unique().size

# see this
amz_rev[amz_rev['description'].str.contains("http")]['description']

# remove everything from "htpp" to ".com" from the description
amz_rev['description'] = amz_rev['description'].str.replace("http.*.com", "")
amz_rev['description'] = amz_rev['description'].str.replace("http.*.html", "")

# remove everything from "www" to ".com" from the description
amz_rev['description'] = amz_rev['description'].str.replace("www.*.com", "")

# remove "http*" from the description
amz_rev['description'] = amz_rev['description'].str.replace("http.*", "")

# count of records with "http" in the description
print("Number records with \"http\" in description:", amz_rev[amz_rev['description'].str.contains("http")]['asin'].unique().size)

Number records with "http" in description: 0


In [131]:
# remove "&nbsp;" from the description
amz_rev['description'] = amz_rev['description'].str.replace("&nbsp", "")

In [132]:
# identify rows with html tags in the description
amz_rev[amz_rev['description'].str.contains("<[^>]*>")]['asin'].unique().size

# identify rows with special characters in the description
amz_rev[amz_rev['description'].str.contains("[^a-zA-Z0-9\s]")]

# remove special characters from the description
amz_rev['description'] = amz_rev['description'].str.replace("[^a-zA-Z0-9\s]", "")


In [133]:
# average length of description
amz_rev['description'].str.len().mean()

741.5782999971789

We are here, finding the mean of the lengths of each individual description in terms of the number of characters.

### Title

In [134]:
# see titles
amz_rev['title']

# create function to clean title column
def clean_title(text):
    # Remove square brackets, single quotes, HTML tags, double quotes, commas, URLs, and CSS styling
    text = re.sub(r"\[|\]|\'|<[^>]*>|\"|,|\\https?://[^\s]+|{[^}]+}", "", text)
    
    # Remove extra whitespace
    text = " ".join(text.split())
    
    # Remove trailing punctuation
    text = re.sub(r"\.$", "", text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# apply function to title column
amz_rev['title'] = amz_rev['title'].apply(clean_title)

# see updated dataframe
amz_rev['title']


563599    steve green: hide em in your heart: 13 bible m...
162328                      christmas eve and other stories
147642                                             forgiven
550234                     santa claus is comin to town vhs
552729                     santa claus is comin to town vhs
                                ...                        
88735     60 glitter gel pens for adult coloring books a...
176940    deego nylon micro cable-002 chefzbest screen p...
256258    vantrue upgraded n1 small dash cam full hd 108...
446622    poison kisses 2: my best kept secret - kindle ...
248118    jricoo nylon braided 2.0 micro usb charging ca...
Name: title, Length: 496258, dtype: object

In [135]:
# check any special characters in title
amz_rev[amz_rev['title'].str.contains("[^a-zA-Z0-9\s]")]

# replace special characters in title
amz_rev['title'] = amz_rev['title'].str.replace("[^a-zA-Z0-9\s]", "")

# count of records with special character in the title
print("Number records with special character in title:", amz_rev[amz_rev['title'].str.contains("[^a-zA-Z0-9\s]")]['asin'].unique().size)


Number records with special character in title: 0


### Brand

In [136]:
# see brand
amz_rev['brand']

# create function to clean brand column
def clean_brand(text):
    # Remove square brackets, single quotes, HTML tags, double quotes, commas, URLs, and CSS styling
    text = re.sub(r"\[|\]|\'|<[^>]*>|\"|,|\\https?://[^\s]+|{[^}]+}", "", text)
    
    # Remove extra whitespace
    text = " ".join(text.split())
    
    # Remove trailing punctuation
    text = re.sub(r"\.$", "", text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text


# apply function to brand column
amz_rev['brand'] = amz_rev['brand'].apply(clean_brand)

# see updated dataframe
amz_rev['brand']

563599                     steve green
162328        trans-siberian orchestra
147642                   don francisco
550234                    fred astaire
552729                    fred astaire
                      ...             
88735                        pita-maci
176940                           deego
256258                         vantrue
446622    visit amazons kia meche page
248118                          jricoo
Name: brand, Length: 496258, dtype: object

In [137]:
# see if any special characters in brand
amz_rev['brand'].str.contains("[^a-zA-Z0-9\s]")

# see rows with special characters
amz_rev[amz_rev['brand'].str.contains("[^a-zA-Z0-9\s]")]

# change "&" to and
amz_rev['brand'] = amz_rev['brand'].str.replace("&", "and")

# change "-" to " "
amz_rev['brand'] = amz_rev['brand'].str.replace("-", " ")

# change "." to " "
amz_rev['brand'] = amz_rev['brand'].str.replace(".", " ")

# replace rest special characters in title
amz_rev['brand'] = amz_rev['brand'].str.replace("[^a-zA-Z0-9\s]", " ")

# see if any products have special characters in brand
print("Number records with special character in brand:", amz_rev[amz_rev['brand'].str.contains("[^a-zA-Z0-9\s]")]['asin'].unique().size)

Number records with special character in brand: 0


### Category



In [138]:
amz_rev.category_x.unique()

array(['movies_and_tv', 'cds_and_vinyl', 'office_products', 'software',
       'toys_and_games', 'electronics', 'video_games',
       'musical_instruments', 'sports_and_outdoors',
       'tools_and_home_improvement', 'pet_supplies', 'digital_music',
       'grocery_and_gourmet_food', 'home_and_kitchen', 'automotive',
       'cell_phones', 'arts_crafts', 'patio_lawn_and_garden',
       'industrial', 'beauty', 'magazine_subscriptions',
       'clothing_shoes_and_jewelry', 'appliances', 'prime_pantry',
       'kindle_store', 'fashion', 'gift_cards', 'luxury_beauty'],
      dtype=object)

In [139]:
# see brand
amz_rev['category_x'].unique()

# replace "_" with ""
amz_rev['category_x'] = amz_rev['category_x'].str.replace("_", "")

# see updated dataframe
amz_rev['category_x'].unique()

array(['moviesandtv', 'cdsandvinyl', 'officeproducts', 'software',
       'toysandgames', 'electronics', 'videogames', 'musicalinstruments',
       'sportsandoutdoors', 'toolsandhomeimprovement', 'petsupplies',
       'digitalmusic', 'groceryandgourmetfood', 'homeandkitchen',
       'automotive', 'cellphones', 'artscrafts', 'patiolawnandgarden',
       'industrial', 'beauty', 'magazinesubscriptions',
       'clothingshoesandjewelry', 'appliances', 'primepantry',
       'kindlestore', 'fashion', 'giftcards', 'luxurybeauty'],
      dtype=object)

## ReviewText

In [140]:
# see brand
amz_rev['reviewText']

# create function to clean brand column
def clean_brand(text):
    # Remove square brackets, single quotes, HTML tags, double quotes, commas, URLs, and CSS styling
    text = re.sub(r"\[|\]|\'|<[^>]*>|\"|,|\\https?://[^\s]+|{[^}]+}", "", text)
    
    # Remove extra whitespace
    text = " ".join(text.split())
    
    # Remove common update notes
    update_regex = r'update \d+/\d+/\d+:'
    text = re.sub(update_regex, '', text)

    # Remove trailing punctuation
    text = re.sub(r"\.$", "", text)

    # remove dates
    text = re.sub(r'\d{2}/\d{2}/\d{2}', '', text)

    
    # Convert to lowercase
    text = text.lower()
    
    return text


# apply function to brand column
amz_rev['reviewText'] = amz_rev['reviewText'].apply(clean_brand)

# see updated dataframe
amz_rev['reviewText']


563599    product worked as advertised and am pleased wi...
162328                       nice to hear this cd beautiful
147642    that one song has to be one of the best in ccm...
550234    i just love these older movies they really are...
552729                    love this movie at christmas time
                                ...                        
88735     one of the gel pens leaked in the case when sh...
176940    love the long long cord. i can plug it is most...
256258    quick and easy review: great small inexpensive...
446622    kia i loved this book. i am so glad that sky g...
248118    only two out of the three we ordered still wor...
Name: reviewText, Length: 496258, dtype: object

In [141]:
# check any special characters in title
amz_rev[amz_rev['reviewText'].str.contains("[^a-zA-Z0-9\s]")]

# replace special characters in title
amz_rev['reviewText'] = amz_rev['reviewText'].str.replace("[^a-zA-Z0-9\s]", "")

# count of records with special character in the title
print("Number records with special character in reviewText:", amz_rev[amz_rev['reviewText'].str.contains("[^a-zA-Z0-9\s]")]['asin'].unique().size)

Number records with special character in reviewText: 0


In [142]:
# see record
display(amz_rev[(amz_rev['reviewTime']=='2012-09-11') & (amz_rev['asin']=='B001YTK3XK')])

# view data 
amz_rev[(amz_rev['reviewTime']=='2012-09-11') & (amz_rev['asin']=='B001YTK3XK')]['reviewText'].values[0]

,reviewerID,reviewerName,reviewTime,asin,title,brand,description,reviewText,category_x,overall
305108,A2LEIANN1UZTHP,brainout,2012-09-11,B001YTK3XK,brother mfc8480dn highperformance laser allino...,brother,the mfc8480dn is a highperformance laser allin...,update this machine is way overpriced now i g...,officeproducts,2.0


'update  this machine is way overpriced now i got mine for maybe 400 will check and write an updated note in the comments it wasnt worth the amount i paid much less the selling price now update  i finally gave up and reinstalled my prior mfc 9700 it scans well this 8480dn cannot scan directly from the machine but only from the computer however at least the 8480dn has a power button along with wire hookup in two ways parallel and usb it also allows you to configure it from the computer without having to use that horrible lcd menu but i gotta tell ya ill never buy another brother printer again they didnt learn any lessons from the insanity of the mfc 9700s frustrating interface update 32213 i upgraded one star for the printer because there is a way to make it do searchable pdf via its paperport function but the scanner itself cant do that you can do tht if you have acrobat its ocr function will also convert any google doc into searchable pdf yay finally i can search shakespeare and gibbo

## <a id='toc2_7_'></a>[Dealing with Categorical Data](#toc0_)

We deal with categorical data (categories). Categorical variables need to be encoded as numerical values for many machine learning algorithms. You can use various encoding techniques such as one-hot encoding, label encoding, or target encoding to transform categorical variables into a numerical format suitable for analysis or modeling.

## <a id='toc2_9_'></a>[Normalizing Data](#toc0_)

We look at normalizing data (price, votes, etc. ) We used min-max. 

In [143]:
# Assuming your ratings data is in a DataFrame called 'df' with a column named 'rating'
min_rating = amz_rev['overall'].min()
max_rating = amz_rev['overall'].max()

# Normalize the ratings to a range from 0 to 1
amz_rev['normalized_rating'] = (amz_rev['overall'] - min_rating) / (max_rating - min_rating)

# see updated dataframe
amz_rev.head(3)

# see summary of normalized ratings
amz_rev['normalized_rating'].describe()


count    496258.000000
mean          0.845894
std           0.271659
min           0.000000
25%           0.750000
50%           1.000000
75%           1.000000
max           1.000000
Name: normalized_rating, dtype: float64

The '`normalized_rating`' column will contain the normalized ratings between 0 and 1, where 0 corresponds to the minimum rating and 1 corresponds to the maximum rating. Note, the '`normalized_rating`' column is created by subtracting the minimum value from each rating and dividing it by the range (maximum value minus minimum value).







## <a id='toc2_10_'></a>[Check Data Imbalance](#toc0_)

We check data balance across categories.

In [144]:
# see distribution of reviews across categories
amz_rev['category_x'].value_counts()

electronics                24941
petsupplies                24928
cellphones                 24905
toysandgames               24875
officeproducts             24870
kindlestore                24837
automotive                 24828
videogames                 24809
musicalinstruments         24782
groceryandgourmetfood      24766
primepantry                24759
homeandkitchen             24736
patiolawnandgarden         24715
toolsandhomeimprovement    24671
artscrafts                 24626
industrial                 24412
sportsandoutdoors          24084
cdsandvinyl                22169
moviesandtv                21242
software                   11911
clothingshoesandjewelry    10752
beauty                      4143
giftcards                   2670
magazinesubscriptions       2250
digitalmusic                 337
appliances                   201
luxurybeauty                  21
fashion                       18
Name: category_x, dtype: int64

In [145]:
# remove categories with less than 100 reviews
amz_rev = amz_rev.groupby('category_x').filter(lambda x: len(x) >= 100)

# see updated dataframe
amz_rev['category_x'].value_counts()


electronics                24941
petsupplies                24928
cellphones                 24905
toysandgames               24875
officeproducts             24870
kindlestore                24837
automotive                 24828
videogames                 24809
musicalinstruments         24782
groceryandgourmetfood      24766
primepantry                24759
homeandkitchen             24736
patiolawnandgarden         24715
toolsandhomeimprovement    24671
artscrafts                 24626
industrial                 24412
sportsandoutdoors          24084
cdsandvinyl                22169
moviesandtv                21242
software                   11911
clothingshoesandjewelry    10752
beauty                      4143
giftcards                   2670
magazinesubscriptions       2250
digitalmusic                 337
appliances                   201
Name: category_x, dtype: int64

***
# <a id='toc3_'></a>[Creating New Dataset and Saving](#toc0_)

We create a new dataset using all the cleaning and processing done above and save it as a csv file called `data_clean.csv`.

In [146]:
# save cleaned data
amz_rev.to_csv("Data/amz_rev_cleaned.csv", index=False)

# stats of cleaned data
print("Shape of cleaned data:", amz_rev.shape)
print("\nNumber of unique products:", amz_rev['asin'].unique().size)
print("\nNumber of unique users:", amz_rev['reviewerID'].unique().size)
print("\nNumber of unique brands:", amz_rev['brand'].unique().size)
print("\nNumber of unique categories:", amz_rev['category_x'].unique().size)
print("\nNumber of reviews per category:\n", amz_rev['category_x'].value_counts())
print("\nColumns available:", amz_rev.columns)

Shape of cleaned data: (496219, 11)

Number of unique products: 251615

Number of unique users: 339625

Number of unique brands: 68503

Number of unique categories: 26

Number of reviews per category:
 electronics                24941
petsupplies                24928
cellphones                 24905
toysandgames               24875
officeproducts             24870
kindlestore                24837
automotive                 24828
videogames                 24809
musicalinstruments         24782
groceryandgourmetfood      24766
primepantry                24759
homeandkitchen             24736
patiolawnandgarden         24715
toolsandhomeimprovement    24671
artscrafts                 24626
industrial                 24412
sportsandoutdoors          24084
cdsandvinyl                22169
moviesandtv                21242
software                   11911
clothingshoesandjewelry    10752
beauty                      4143
giftcards                   2670
magazinesubscriptions       2250
digita

In [ ]:
# check for null values in the data for each column
print("Null values in each column:\n", round(all_revs_meta.isnull().sum()/len(all_revs_meta)*100,2))